This is code that was previously used, but is no longer used.

### Loop for training ensemble models
```python

plot_while_training = False

def train_mlp(model, train_embeddings, train_labels, validation_embeddings, validation_labels, epochs=100, batch_size=2048, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    
    training_losses = []
    validation_accuracies = []
    
    # Create DataLoader for training data with shuffling
    train_dataset = TensorDataset(train_embeddings, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        
        for batch_embeddings, batch_labels in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_embeddings)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(train_loader)
        training_losses.append(avg_loss)
        
        # Evaluate on validation set
        model.eval()
        with torch.no_grad():
            val_outputs = model(validation_embeddings)
            val_loss = criterion(val_outputs, validation_labels)
            _, predicted = torch.max(val_outputs, 1)
            accuracy = (predicted == validation_labels).sum().item() / validation_labels.size(0)
            validation_accuracies.append(accuracy)
        
        print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Validation Loss: {val_loss.item():.4f}, Validation Accuracy: {accuracy*100:.2f}%')
    
    # Plotting training loss and validation accuracy
    if plot_while_training:
        epochs_range = range(1, epochs+1)
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, training_losses, label='Training Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title('Training Loss')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, [acc * 100 for acc in validation_accuracies], label='Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy (%)')
        plt.title('Validation Accuracy')
        plt.legend()
        
        plt.tight_layout()
        plt.show()
    
    return training_losses, validation_accuracies

# Function to evaluate a single MLP
def evaluate_mlp(model, embeddings, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(embeddings)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == labels).sum().item() / labels.size(0)
    return accuracy, predicted.cpu().numpy()

# Function to evaluate an ensemble of MLPs
def evaluate_ensemble(ensemble, embeddings, labels, num_models=None):
    ensemble_outputs = torch.zeros((embeddings.size(0), output_size)).to(device)
    if num_models is None:
        num_models = len(ensemble)
    else:
        num_models = min(num_models, len(ensemble))
    
    for model in ensemble[:num_models]:
        model.eval()
        with torch.no_grad():
            outputs = model(embeddings)
            ensemble_outputs += outputs
    
    _, predicted = torch.max(ensemble_outputs, 1)
    accuracy = (predicted == labels).sum().item() / labels.size(0)
    return accuracy, predicted.cpu().numpy()

# Training and evaluating the ensemble
validation_accuracies_per_model = []

for idx, model in enumerate(ensemble, 1):
    print(f"\nTraining model {idx}/{ensemble_size}")
    val_accuracy = train_mlp(model, train_embeddings_tensor, train_labels_tensor, validation_embeddings_tensor, validation_labels_tensor)
    validation_accuracies_per_model.append(val_accuracy[1])

# Plot validation accuracies for each ensemble member
plt.figure()
plt.plot(range(1, ensemble_size+1), [acc[-1] * 100 for acc in validation_accuracies_per_model], marker='o')
plt.xlabel('Ensemble Member')
plt.ylabel('Validation Accuracy (%)')
plt.title('Validation Accuracy per Ensemble Member')
plt.xticks(range(1, ensemble_size+1))
plt.grid(True)
plt.show()

# Evaluate the ensemble on the validation set incrementally
ensemble_sizes = range(1, ensemble_size + 1)
ensemble_accuracies = []

for n_models in ensemble_sizes:
    accuracy, _ = evaluate_ensemble(ensemble, validation_embeddings_tensor, validation_labels_tensor, num_models=n_models)
    ensemble_accuracies.append(accuracy)
    print(f'Ensemble Size: {n_models}, Validation Accuracy: {accuracy * 100:.2f}%')

# Plot ensemble accuracy vs ensemble size
plt.figure()
plt.plot(ensemble_sizes, [acc * 100 for acc in ensemble_accuracies], marker='o')
plt.xlabel('Number of Models in Ensemble')
plt.ylabel('Validation Accuracy (%)')
plt.title('Ensemble Validation Accuracy vs Ensemble Size')
plt.xticks(ensemble_sizes)
plt.grid(True)
plt.show()

# Additional evaluation metrics for the full ensemble
ensemble_accuracy, ensemble_predicted = evaluate_ensemble(ensemble, validation_embeddings_tensor, validation_labels_tensor)
print(f'\nFull Ensemble Validation Accuracy: {ensemble_accuracy * 100:.2f}%')

# Classification report and confusion matrix
validation_labels_np = validation_labels_tensor.cpu().numpy()

print("\nClassification Report for Full Ensemble:")
print(classification_report(validation_labels_np, ensemble_predicted, target_names=['Negative', 'Positive']))

print("\nConfusion Matrix for Full Ensemble:")
print(confusion_matrix(validation_labels_np, ensemble_predicted))

```
